In [1]:
from datetime import date, timedelta
from itertools import combinations
import pandas as pd
import warnings

# Suppress runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# ✅ Assuming these modules exist in your local environment
from strategies.macd import MACDStrategy
from strategies.bollinger import BollingerBandsStrategy
from strategies.cci import CCI_Strategy
from strategies.adx import ADXStrategy
from strategies.obv import OBVStrategy

from utils import (
    combine_signals,
    parameter_cache,
    optimize_macd,
    optimize_bollinger_bands,
    optimize_cci,
    optimize_adx
)

from data_loader import fetch_data
from strategy_processor import StrategyProcessor
from backtesting_wrapper import BacktestingWrapper

# Configuration
ticker = "AAPL"
start_date = "2024-04-01"
end_date = "2025-04-01"
interval = "1h"
selected_strategies = ["MACD", "BB", "CCI", "ADX", "OBV"]

# Fetch data
data = fetch_data(ticker, start_date, end_date, interval)

# Apply selected strategies
all_processed_data = data.copy()
for strategy_name in selected_strategies:
    strategy = None
    if strategy_name == "MACD":
        params = optimize_macd(data, 5, 30)
        strategy = MACDStrategy(
            fast_length=params["fast_length"],
            slow_length=params["slow_length"]
        )
    elif strategy_name == "BB":
        params = optimize_bollinger_bands(data, 5, 25, 1, 3)
        strategy = BollingerBandsStrategy(
            length=params["length"],
            std_dev_multiplier=params["std_dev_multiplier"]
        )
    elif strategy_name == "CCI":
        params = optimize_cci(data, 5, 25)
        strategy = CCI_Strategy(
            length=params["length"]
        )
    elif strategy_name == "ADX":
        params = optimize_adx(data, 5, 25, 20, 40)
        strategy = ADXStrategy(
            length=params["length"],
            threshold=params["threshold"]
        )
    elif strategy_name == "OBV":
        strategy = OBVStrategy()

    processor = StrategyProcessor(strategy, data)
    result = processor.process_data()
    all_processed_data[f'BuySignal{strategy_name}'] = result['BuySignal']
    all_processed_data[f'SellSignal{strategy_name}'] = result['SellSignal']

# Combine signals
all_processed_data = combine_signals(all_processed_data, selected_strategies)

# Backtest all combinations
results = []
for r in range(1, len(selected_strategies) + 1):
    for combo in combinations(selected_strategies, r):
        # Skip OBV-only combinations
        if combo == ("OBV",):
            continue

        df = all_processed_data.copy()
        if len(combo) == 1:
            s = combo[0]
            df["CommonBuySignal"] = df[f"BuySignal{s}"]
            df["CommonSellSignal"] = df[f"SellSignal{s}"]
        else:
            df = combine_signals(df, list(combo))

        bt = BacktestingWrapper(strategy=None)
        stats = bt.backtest(df)
        metrics = bt.extract_statistics(stats)
        results.append({
            "Strategies": " + ".join(combo),
            "Return [%]": metrics['Return [%]'],
            "Trades": metrics['Number of Trades'],
            "Best Trade [$]": metrics['Best Trade [$]'],
            "Worst Trade [$]": metrics['Worst Trade [$]']
        })

# Show results
df_results = pd.DataFrame(results)
df_results.sort_values("Return [%]", ascending=False, inplace=True)

df_results


C:\Users\rayyo\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

📥 Fetching 1h data for AAPL from 2024-04-01 to 2025-04-01...


[*********************100%***********************]  1 of 1 completed



[MACD] Top 5 Parameter Combinations:
Fast Length  Slow Length  Return [%]
20           22           9.22      
18           23           6.88      
14           29           4.94      
16           26           3.62      
13           24           1.38      

[BollingerBands] Top 5 Parameter Combinations:
Length     Std Dev Multiplier   Return [%]
13         2.6                  52.62     
13         2.5                  41.48     
13         2.7                  40.02     
9          2.2                  39.12     
11         2.5                  37.98     

[CCI] Top 5 Parameter Combinations:
Length     Return [%]
10         24.44     
11         22.10     
12         18.58     
9          16.56     
8          15.94     

[ADX] Top 5 Parameter Combinations:
Length     Threshold    Return [%]
25         40           -0.81     
24         38           -9.65     
22         38           -10.97    
25         39           -11.11    
24         39           -11.83    


,Strategies,Return [%],Trades,Best Trade [$],Worst Trade [$]
8,BB + CCI,52.621157,11,1848.797846,-63.552016
1,BB,52.621157,11,1848.797846,-63.552016
11,CCI + ADX,34.027388,4,2074.519564,-255.967610
23,CCI + ADX + OBV,34.027388,4,2074.519564,-255.967610
2,CCI,24.438158,45,763.904236,-635.866207
12,CCI + OBV,15.192131,22,773.305971,-268.899001
19,MACD + ADX + OBV,9.379599,15,1192.758693,-531.757489
6,MACD + ADX,9.379599,15,1192.758693,-531.757489
0,MACD,9.215046,88,816.998779,-442.601363
18,MACD + CCI + OBV,0.000000,0,NaN,NaN
